# Fast AI pre-processing

In [1]:
from fastai.text.all import *

In [2]:
import dill

In [3]:
def printMemoryReport():
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

## Train on Movie dialogs

In [4]:
cwd = 'D:/Pytorch Data/cornell_movie_dialogs/'

In [5]:
data = pd.read_csv(cwd + 'unaltered_movie_dialogs.csv')

In [6]:
# The warning means nothing and the fast ai should be more transparent about needing this
data['is_valid'] = False
data['is_valid'].loc[data.index[:(len(data) // 10)]] = True

D:\Anaconda3\envs\fastai\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
data.columns

Index(['Unnamed: 0', 'input', 'truth', 'is_valid'], dtype='object')

## Try Combing Each line with the truth

In [8]:
data['combined'] = data['input'] + ' : ' +  data['truth']

In [9]:
dls_lm = TextDataLoaders.from_df(data, path=cwd, text_col='combined', is_lm=True, 
                                 valid_col='is_valid', max_vocab = 10000, seq_len = 10, num_workers = 0)

D:\Anaconda3\envs\fastai\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [10]:
dls_lm.show_batch(max_n=3)

,text,text_
0,xxbos xxmaj the < u > thinking < / u,xxmaj the < u > thinking < / u >
1,"'ve got to find shelter soon , and so do","got to find shelter soon , and so do we"
2,it 's nothing . \n▁ : xxmaj my xxunk instinct,'s nothing . \n▁ : xxmaj my xxunk instinct tells


In [11]:
outfile = open(cwd + 'movie_dialog_dlm_10000.pickle','wb')
dill.dump(dls_lm, outfile)
outfile.close()

In [12]:
del dls_lm

In [13]:
printMemoryReport()

GeForce GTX 1060 with Max-Q Design
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [8]:
dls_lm = dill.load(open(cwd + 'movie_dialog_dlm_10000.pickle', 'rb'))

In [10]:
printMemoryReport()

GeForce GTX 1060 with Max-Q Design
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [9]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult = 0.3,
    metrics = [accuracy, Perplexity()], 
).to_fp16()

In [12]:
learn.fine_tune(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.797482,3.675055,0.335575,39.450829,26:31


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.266187,3.312304,0.373925,27.448301,32:52


In [13]:
learn.save('1epoch_combined_10000')

Path('D:/Pytorch Data/cornell_movie_dialogs/models/1epoch_combined_10000.pth')

In [14]:
learn.load('1epoch_combined_10000')

In [15]:
learn.unfreeze()
learn = learn.to_fp32()
learn.fit_one_cycle(5, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.277561,3.402505,0.361776,30.039267,32:17
1,3.263827,3.368766,0.369343,29.042654,33:07
2,3.170397,3.332192,0.375224,27.999662,33:17
3,3.013740,3.321114,0.378257,27.691177,34:12
4,2.873098,3.344338,0.377580,28.341812,34:40


In [16]:
learn.save('6epoch_combined_10000')

Path('D:/Pytorch Data/cornell_movie_dialogs/models/6epoch_combined_10000.pth')

In [10]:
learn.load('6epoch_combined_10000')

In [12]:
input_text   = 'Hello there my good friend. :'
nr_words     = 30
nr_sentences = 6
[learn.predict(input_text, nr_words, temperature = 0.5) for _ in range(nr_sentences)]

["Hello there my good friend . : What is it ? \n▁ : It 's Tom . What about Mr . Carpenter ? \n▁ : He 's a good",
 "Hello there my good friend . : What 's your name ? \n▁ : What are you doing here ? You know , i just do n't like to talk about it .",
 "Hello there my good friend . : Hi . \n▁ : Hi . We 're not going to give up your life for a second . You 're going to be working",
 "Hello there my good friend . : Where 's Bill ? \n▁ : He 's in the Army . It 's too late for that . \n▁ : I 'm",
 "Hello there my good friend . : What do you want to do ? \n▁ : I 'm sorry . I 'm sorry . It 's just that I 'm really sorry",
 "Hello there my good friend . : Oh , yes , I 'm Dr . Very good . The Doctor . \n▁ : The Doctor ? No"]

In [13]:
input_text   = 'What do you want? :'
nr_words     = 12
nr_sentences = 6
[learn.predict(input_text, nr_words, temperature = 0.5) for _ in range(nr_sentences)]

['What do you want ? : i want to die here . For all i know you',
 "What do you want ? : You know , I 'm not going to tell you",
 "What do you want ? : I 'm going to Paris . Get a job",
 "What do you want ? : You know i do n't . \n▁ : You 're",
 'What do you want ? : The United States is a state of the dead',
 "What do you want ? : You 're a fucking liar . You 're a fuckin"]

In [14]:
input_text   = 'Who are you? :'
nr_words     = 50
nr_sentences = 6
[learn.predict(input_text, nr_words, temperature = 0.5) for _ in range(nr_sentences)]

["Who are you ? : You 're a doctor . You 're a doctor , Dr . Towler . You were a doctor . You were a doctor of medical duty . And I 'm gon na get you a new one . \n▁ :",
 "Who are you ? : What are you , Mr . White ? \n▁ : My name is John . I 'm John Doe Clubs . We 're gon na be together . And we 're going to Florida . We 're",
 "Who are you ? : You 're my friend . You 're here . You 're here . I 'm not sure . i just think it 's pretty obvious . \n▁ : Well , you know , I 'm not the kind of guy who 's very",
 "Who are you ? : I 'm Bob Woodward of the Post . I 'm Bob Woodward of the Post Office , Post Office , Post Office , Post Office , Post Office , Post Office",
 "Who are you ? : Who are you ? Who are you ? What ? \n▁ : The Red Sox are n't a game . I 'm thinking of Blue Max . You do n't know . \n▁ : There 's a",
 "Who are you ? : What do you want ? \n▁ : You know … I 'm not a man . I 'm a man . I 'm a man . What 's the matter ? \n▁ : You 're crazy . You have to"]

In [15]:
input_text   = 'Hello. :'
nr_words     = 50
nr_sentences = 6
[learn.predict(input_text, nr_words, temperature = 0.5) for _ in range(nr_sentences)]

["Hello . : Where you been ? ! Where are you ? ! \n▁ : I 'm here . Do n't you know I 'm here ? You 're not going to be all right . I 'll have a good time . \n▁ :",
 "Hello . : How 's that ? \n▁ : Good . Good . We 're not gon na get any information . We 've got to get out of here . \n▁ : You 're not going to get hurt . I 'm sorry",
 "Hello . : It 's Hi . It 's Hi . How 's the new dress ? \n▁ : It 's fine . What 's that ? \n▁ : You 're not going to be around long enough to get it back .",
 "Hello . : Hi . \n▁ : Hi . Oh , Jesus . \n▁ : I 'm sorry . I 'm just a little nervous . I 'm not sure what i can do about it . You 're a lucky man ,",
 "Hello . : Hi . \n▁ : Hello . You 're a little too young . \n▁ : I 'm just a little bit nervous . What 's that ? \n▁ : That 's the one . Do n't you think it 's",
 "Hello . : Hello , I 'm Mr . Pink . \n▁ : Hi . You know , I 've never seen you before . \n▁ : I 'm not sure . But i like the idea . The thing is ,"]

Note that the conversation change token can be used to say that the network thinks that the line is finished. This can be used to avoid cutting the sentence off in the middle of a line.

## Train on Pokemon Story

In [2]:
cwd = 'D:/Websites/StoryWebsiteChatBots/Data/'

In [3]:
data = pd.read_csv(cwd + 'pokemon_story.csv')

In [4]:
data['is_valid'] = False
data['is_valid'].loc[data.index[:(len(data) // 10)]] = True

C:\Users\berta\Anaconda3\envs\fastai\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [5]:
data['combined'] = data['input'] + ' : ' +  data['truth']

In [ ]:
dls_lm = TextDataLoaders.from_df(data, path=cwd, text_col='combined', is_lm=True, 
                                 valid_col='is_valid', max_vocab = 20000, seq_len = 10, num_workers = 0)

In [ ]:
dls_lm.show_batch(max_n=3)

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult = 0.3,
    metrics = [accuracy, Perplexity()], 
)

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [11]:
learn.save('1epochpokemon')

Path('D:/Websites/StoryWebsiteChatBots/Data/models/1epochpokemon.pth')

In [12]:
learn.load('1epochpokemon')

In [13]:
learn.unfreeze()
learn.fit_one_cycle(5, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.804402,4.637709,0.256711,103.307365,34:00
1,3.597514,4.635091,0.262848,103.037277,34:50
2,3.281948,4.752848,0.258904,115.913902,35:38
3,2.859172,4.920225,0.255779,137.033401,35:35
4,2.633348,5.018977,0.252899,151.256516,35:36


In [14]:
learn.save_encoder('pokemon_encoded_model')

In [20]:
input_text   = 'Ash crept through the cave and threw a pokeball . :'
nr_words     = 50
nr_sentences = 1
[learn.predict(input_text, nr_words, temperature = 0.5) for _ in range(nr_sentences)]

["Ash crept through the cave and threw a pokeball . : it was a little hard to concentrate but he was already moving again a second or so later and the whole thing collapsed in a moment . then the ground exploded upwards . i guess that 's a good sign ash agreed . : it 's a pity we"]

## Including speech marks and one sentence

In [2]:
cwd = 'D:/Websites/StoryWebsiteChatBots/Data/'

In [3]:
data = pd.read_csv(cwd + 'pokemon_story_one_setence.csv')

In [4]:
data['combined'] = data['input'] + ' : ' +  data['truth']

In [5]:
data['is_valid'] = False
data['is_valid'].loc[data.index[:(len(data) // 10)]] = True

C:\Users\berta\Anaconda3\envs\fastai\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
dls_lm = TextDataLoaders.from_df(data, path=cwd, text_col='combined', is_lm=True, 
                                 valid_col='is_valid', max_vocab = 10000, seq_len = 10, num_workers = 0)

C:\Users\berta\Anaconda3\envs\fastai\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [8]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult = 0.3,
    metrics = [accuracy, Perplexity()], 
)

In [9]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.852951,4.257229,0.305129,70.614029,17:16


In [10]:
learn.save('1epochpokomoon')

Path('D:/Websites/StoryWebsiteChatBots/Data/models/1epochpokomoon.pth')

In [11]:
learn.load('1epochpokomoon')

In [12]:
learn.unfreeze()
learn.fit_one_cycle(5, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.583581,4.208026,0.311394,67.223740,17:25
1,3.423593,4.181067,0.316788,65.435638,17:30
2,3.176568,4.212312,0.318875,67.512428,17:37
3,2.894980,4.319005,0.316744,75.113815,17:41
4,2.680932,4.414442,0.313320,82.635719,17:49


In [13]:
learn.save_encoder('pokomoon_encoded_model')

In [41]:
nr_words = 50

In [44]:
def writeSentence(input_text, temperature = 0.5):
    input_text  += ' : '
    return_line = learn.predict(input_text, nr_words, temperature = 0.5)
    return_line = return_line[return_line.index(":") + 1:] # should be substr
    return_line = return_line[:return_line.index(":")]
    return return_line

In [45]:
writeSentence("Ash wandered into the cave and saw a geodude.", 0.5)

' " oh no " " what ? " " that \'s right " brock confirmed . '

In [49]:
writeSentence("Brock blocked the entrance to the cave with his car.", 0.75)

' " what \'s the plan ? " " yeah " ash agreed . '

In [52]:
writeSentence("Geodude suddenly exploded.", 0.75)

' " that \'s a new one on me " lucario said . " " i think i \'m getting a headache " pikachu said . '

In [55]:
writeSentence("pikachu used thunderbolt on geodude.", 0.75)

' the impact drove him backwards a little and he landed in a crater . " " i think it was " ash replied . '

In [56]:
writeSentence("lucario jumped in the air.", 0.75)

' he was n\'t sure what the best way to learn but it was an excellent example of his art . " i \'m sure you can help . '

In [60]:
writeSentence("brock was then arrested.", 0.75)

' " i guess i \'ll have to explain this one " he decided . " " that \'s right " brock confirmed . '

In [71]:
writeSentence("officer jenny walked in.", 0.5)

' " i think that \'s a good sign " she said . " " i \'m not sure " may admitted . '